In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import sys
import os
import pandas as pd
import numpy as np
sys.path.append('/content/gdrive/MyDrive')

**- Install Requirements**

In [ ]:
!pip install  transformers datasets evaluate accelerate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=75019a79997d4502dd3f48d1c8aade8086981221437988382bc17175b94d9aeb
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a9

In [ ]:
import os
import sys

from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification
)

### Load Data

In [ ]:
data = load_dataset("csv", data_files="/content/gdrive/MyDrive/final_tokens_labels.csv")
train_test_datasets = data['train'].train_test_split(train_size=0.8, seed=42)

In [ ]:
train_test_datasets

DatasetDict({
    train: Dataset({
        features: ['Token', 'Label'],
        num_rows: 121209
    })
    test: Dataset({
        features: ['Token', 'Label'],
        num_rows: 30303
    })
})

In [ ]:
data['train']['Token'][:10]

['የሞተ', 'ቆዳን', 'እንዲሁም', 'ቆሻሻን', 'ለማፅዳት', 'ተመራጭ', 'ዋጋ', '200', 'ብር', 'ውስን']

In [ ]:
data['train']['Label'][:10]

['O', 'O', 'O', 'O', 'O', 'O', 'B-PRICE', 'I-PRICE', 'I-PRICE', 'O']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
labels = list(set(data['train']['Label']))
label_mapping = {label: i for i, label in enumerate(labels)}

In [ ]:
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(example['Token'], truncation=True, padding='max_length', max_length=128)

    # Initialize labels with -100 for tokens that should be ignored
    labels = [-100] * len(tokenized_inputs['input_ids'])

    # Align labels with tokens
    label_index = label_mapping[example["Label"]]

    # Fill labels according to the example
    for i in range(len(tokenized_inputs['input_ids'])):
        # Assuming example['Token'] is not empty and corresponds to the label
        if example['Token'] == example['Token']:
            labels[i] = label_index

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# Tokenize the dataset
tokenized_dataset = train_test_datasets.map(tokenize_and_align_labels)

Map:   0%|          | 0/121209 [00:00<?, ? examples/s]

Map:   0%|          | 0/30303 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['Token', 'Label', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 121209
    })
    test: Dataset({
        features: ['Token', 'Label', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 30303
    })
})

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    weight_decay=0.1,
    evaluation_strategy="steps",
    logging_steps=100,
    eval_steps=500,
    save_steps=500,
    # save_strategy="epoch",
    # logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    seed=42,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Load the model
model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=len(labels))

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Initialize the data collator
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True, return_tensors="pt")

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # Flatten the arrays and ensure they are of type int32
    return metric.compute(predictions=predictions.flatten().astype(np.int32), references=labels.flatten().astype(np.int32))

In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
# Train the model
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
500,0.931800,0.932662,0.715606
1000,0.777400,0.795324,0.765964
1500,0.822600,0.783728,0.765964
2000,0.899000,0.935352,0.715606
2500,0.921600,0.934221,0.715606
3000,0.941800,0.931584,0.715606
3500,0.891600,0.934255,0.715606


TrainOutput(global_step=3788, training_loss=0.8955632917732472, metrics={'train_runtime': 2607.3449, 'train_samples_per_second': 46.488, 'train_steps_per_second': 1.453, 'total_flos': 7918094402000640.0, 'train_loss': 0.8955632917732472, 'epoch': 1.0})

In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Evaluation results: {'eval_loss': 0.7953243255615234, 'eval_accuracy': 0.7659637659637659, 'eval_runtime': 110.1102, 'eval_samples_per_second': 275.206, 'eval_steps_per_second': 34.402, 'epoch': 1.0}


In [ ]:
# Save the fine-tuned model and tokenizer
trainer.save_model('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/sentencepiece.bpe.model',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')